In [3]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

### Constantes

**Acá se deben cambiar los paths a las carpetas docs-raw-texts y queries-raw-texts**

In [4]:
DOCS_RAW_DIRECTORY_PATH = "./docs-raw-texts/"
QUERIES_DIRECTORY_PATH = "./queries-raw-texts/"

### Extracción

In [5]:
def parse_naf_document(filepath: str) -> dict:
    """
    Parsea un archivo NAF y extrae el título, el contenido y el publicId.

    Args:
        filepath: La ruta al archivo NAF.

    Returns:
        Un diccionario con el título, el contenido, el publicId y el raw_text, o None si hay un error.
    """
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()

        # Extraer el título
        title = root.find('.//fileDesc').get('title') if root.find('.//fileDesc') is not None else None

        # Extraer el contenido (raw text)
        content = root.find('.//raw').text if root.find('.//raw') is not None else None

        # Extraer el publicId
        public_id = root.find('.//public').get('publicId') if root.find('.//public') is not None else None

        # Crear raw_text
        raw_text = f"{title}\n{content}" if title and content else content or title
        return {'ID': public_id, 'title': title, 'content': content, 'raw_text': raw_text}

    except (ET.ParseError, FileNotFoundError, AttributeError) as e:
        print(f"Error al parsear {filepath}: {e}")
        return None


def ingest_naf_documents(directory: str) -> pd.DataFrame:
    """
    Ingesta documentos NAF de un directorio y crea un DataFrame de pandas.

    Args:
        directory: El directorio que contiene los archivos NAF.

    Returns:
        Un DataFrame de pandas con las columnas 'title' y 'content', o None si hay un error.
    """
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".naf"):
            filepath = os.path.join(directory, filename)
            document_data = parse_naf_document(filepath)
            if document_data:
                data.append(document_data)

    if not data:
        print("No se encontraron archivos NAF válidos en el directorio.")
        return None

    df = pd.DataFrame(data)
    return df

In [6]:
df_documents = ingest_naf_documents(DOCS_RAW_DIRECTORY_PATH)

if df_documents is not None:
    print(df_documents.head())
    print(100*"*")
    print(f"Número de documentos cargados: {len(df_documents)}")

     ID                                              title  \
0  d102  William Makepeace Thackeray’s deft Skewering o...   
1  d035  Nicholas Culpeper and the Complete Herbs of En...   
2  d321                    Aviation Pioneer Harriet Quimby   
3  d094                   The Plays of George Bernard Shaw   
4  d014  Hermann ‘Klecks’ Rorschach and his Eponymous Test   

                                             content  \
0  William Makepeace Thackeray’s deft Skewering o...   
1  Nicholas Culpeper and the Complete Herbs of En...   
2  Aviation Pioneer Harriet Quimby.\n\nHarriet Qu...   
3  The Plays of George Bernard Shaw.\n\nGeorge Be...   
4  Hermann ‘Klecks’ Rorschach and his Eponymous T...   

                                            raw_text  
0  William Makepeace Thackeray’s deft Skewering o...  
1  Nicholas Culpeper and the Complete Herbs of En...  
2  Aviation Pioneer Harriet Quimby\nAviation Pion...  
3  The Plays of George Bernard Shaw\nThe Plays of...  
4  Hermann ‘Kle

In [7]:
df_documents

,ID,title,content,raw_text
0,d102,William Makepeace Thackeray’s deft Skewering o...,William Makepeace Thackeray’s deft Skewering o...,William Makepeace Thackeray’s deft Skewering o...
1,d035,Nicholas Culpeper and the Complete Herbs of En...,Nicholas Culpeper and the Complete Herbs of En...,Nicholas Culpeper and the Complete Herbs of En...
2,d321,Aviation Pioneer Harriet Quimby,Aviation Pioneer Harriet Quimby.\n\nHarriet Qu...,Aviation Pioneer Harriet Quimby\nAviation Pion...
3,d094,The Plays of George Bernard Shaw,The Plays of George Bernard Shaw.\n\nGeorge Be...,The Plays of George Bernard Shaw\nThe Plays of...
4,d014,Hermann ‘Klecks’ Rorschach and his Eponymous Test,Hermann ‘Klecks’ Rorschach and his Eponymous T...,Hermann ‘Klecks’ Rorschach and his Eponymous T...
...,...,...,...,...
326,d098,Friedrich Bessel and the Distances of Stars,Friedrich Bessel and the Distances of Stars.\n...,Friedrich Bessel and the Distances of Stars\nF...
327,d043,Henry Cavendish and the Weight of the Earth,Henry Cavendish and the Weight of the Earth.\n...,Henry Cavendish and the Weight of the Earth\nH...
328,d208,ENIAC – The First Computer Introduced Into Public,ENIAC – The First Computer Introduced Into Pub...,ENIAC – The First Computer Introduced Into Pub...
329,d261,Andrea Cesalpino and the Classification of Plants,Andrea Cesalpino and the Classification of Pla...,Andrea Cesalpino and the Classification of Pla...


### Preprocesamiento

Este pipeline de preprocesamiento transforma el texto crudo en un formato limpio y estructurado, listo para el análisis de NLP. Se implementó usando la librería *nltk*. La secuencia de pasos es la siguiente:

`INPUT` → Tokenización → Eliminación de Stop Words → Eliminación de Puntuación → Stemming → Minúsculas → `OUTPUT`

In [8]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess_text(text: str, language:str ='english') -> list:
    # Tokenización
    tokens = word_tokenize(text)
    
    # Remover stopwords
    stop_words = set(stopwords.words(language))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Remover signos de puntuación (solo tokens alfanuméricos)
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    tokens = tokenizer.tokenize(' '.join(tokens))
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Pasar a minúsculas
    text = text.lower()
    
    return tokens

[nltk_data] Downloading package punkt to /home/schica/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/schica/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/schica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_documents['tokens'] = df_documents['raw_text'].apply(lambda x: preprocess_text(x, language='english'))

print(df_documents[['raw_text', 'tokens']].head())

                                            raw_text  \
0  William Makepeace Thackeray’s deft Skewering o...   
1  Nicholas Culpeper and the Complete Herbs of En...   
2  Aviation Pioneer Harriet Quimby\nAviation Pion...   
3  The Plays of George Bernard Shaw\nThe Plays of...   
4  Hermann ‘Klecks’ Rorschach and his Eponymous T...   

                                              tokens  
0  [william, makepeac, thackeray, deft, skewer, h...  
1  [nichola, culpep, complet, herb, england, nich...  
2  [aviat, pioneer, harriet, quimbi, aviat, pione...  
3  [the, play, georg, bernard, shaw, the, play, g...  
4  [hermann, kleck, rorschach, eponym, test, herm...  


### Índice invertido y Búsqueda binaria

**ENUNCIADO: [10p] Cree su propia implementación del índice invertido usando los 331 documentos en el conjunto de datos.**

Los pasos para construir el índice invertido son descritos a continuación (Manrique, R. Clase de PLN, 2025.)

- Paso 1: Preprocesamiento.
- Paso 2 Secuencia de Tokens: Construir una secuencia de tokens asociada al documento del cual fue extraído.
- PASO 3: Se ordena la secuencia de tokens.
- PASO 4: múltiples entradas de un mismo termino en un mismo documento se combinan.
- PASO 5 Indexar: Dividir en un diccionario y postings, y agregar la información de la frecuencia a
nivel de documento.

In [10]:
def build_inverted_index(df, id_col='ID', tokens_col='tokens'):
    """
    Construye un índice invertido con postings, frecuencia de documento y frecuencia de término por documento usando solo pandas.
    Args:
        df: DataFrame con los documentos.
        id_col: nombre de la columna con el ID del documento.
        tokens_col: nombre de la columna con la lista de tokens procesados.
    Returns:
        DataFrame con las columnas 'term', 'postings', 'doc_freq', 'term_freqs'.
    """
    # Explode para tener una fila por token y documento
    exploded = df[[id_col, tokens_col]].explode(tokens_col)

    # Calcular frecuencia de término por documento
    term_doc_freq = exploded.groupby([tokens_col, id_col]).size().reset_index(name='freq')

    # Construir diccionario {doc_id: freq} para cada término
    term_freqs = term_doc_freq.groupby(tokens_col).apply(
        lambda g: dict(zip(g[id_col], g['freq']))
    )

    # Elimina duplicados para postings y doc_freq
    exploded_nodup = exploded.drop_duplicates()

    grouped = exploded_nodup.groupby(tokens_col)[id_col].agg(list)
    doc_freq = exploded_nodup.groupby(tokens_col)[id_col].nunique()

    index_df = pd.DataFrame({
        'term': grouped.index,
        'postings': grouped.values,
        'doc_freq': doc_freq.values,
        'term_freqs': term_freqs.values
    })

    return index_df

In [11]:
inverted_index = build_inverted_index(df_documents, id_col='ID', tokens_col='tokens')

## Ver df resultante
inverted_index

/tmp/ipykernel_22039/2385056869.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  term_freqs = term_doc_freq.groupby(tokens_col).apply(


,term,postings,doc_freq,term_freqs
0,a,"[d102, d035, d116, d071, d250, d114, d156, d18...",129,"{'d003': 2, 'd004': 1, 'd006': 1, 'd011': 1, '..."
1,aachen,"[d252, d139, d161]",3,"{'d139': 1, 'd161': 2, 'd252': 1}"
2,aazv,[d156],1,{'d156': 1}
3,ab,[d224],1,{'d224': 1}
4,abadon,[d062],1,{'d062': 1}
...,...,...,...,...
12767,zurich,"[d014, d143, d113, d112, d030, d047, d059, d21...",11,"{'d014': 2, 'd030': 2, 'd047': 2, 'd059': 1, '..."
12768,zuse,"[d211, d202]",2,"{'d202': 1, 'd211': 20}"
12769,zwicki,[d253],1,{'d253': 17}
12770,zworykin,"[d071, d068]",2,"{'d068': 1, 'd071': 4}"


**ENUNCIADO: [10p] Cree una función que lea el índice invertido y calcule consultas booleanas mediante el algoritmo de mezcla. El algoritmo de mezcla debe ser capaz de calcular: AND, y NOT.**

Los pasos para construir las consultas booleanas, mediante el algoritmo de mezcla, son:

- Paso 1: Preprocesamiento. Se utiliza el mismo con el que se construyó el índice invertido.
- Paso 2: Localizar los documentos donde se encuentran cada uno de los términos de la búsqueda.
- OPCIONAL: Se crea la lista de todos los IDs de documentos (solo necesario para NOT)
- PASO 3: Se ejecuta la operación de mezcla.

In [12]:
def boolean_query(index_df, operation, terms, all_doc_ids=None, language='english'):
    """
    Realiza consultas booleanas (AND, OR, NOT) sobre el índice invertido en formato DataFrame.
    Preprocesa los términos de consulta igual que los documentos.
    Args:
        index_df: DataFrame del índice invertido con columnas 'term', 'postings', 'doc_freq'.
        operation: 'AND', 'OR', o 'NOT'. Se incluye OR a pesar de no estar en el enunciado.
        terms: Lista de términos a consultar (pueden ser frases).
        all_doc_ids: Lista de todos los IDs de documentos (solo necesario para NOT).
        language: Idioma para el preprocesamiento.
    Returns:
        DataFrame con los IDs de documentos que cumplen la consulta.
    """
    # Preprocesar términos de consulta
    processed_terms = []
    for term in terms:
        processed = preprocess_text(term, language=language)
        processed_terms.extend(processed)
    # Eliminar duplicados en la consulta
    processed_terms = list(set(processed_terms))

    # Recuperar postings para cada término
    postings_lists = []
    for term in processed_terms:
        row = index_df[index_df['term'] == term]
        if not row.empty:
            postings_lists.append(set(row.iloc[0]['postings']))
        else:
            postings_lists.append(set())

    if operation == 'AND':
        result = set.intersection(*postings_lists) if postings_lists else set()
    elif operation == 'OR':
        result = set.union(*postings_lists) if postings_lists else set()
    elif operation == 'NOT':
        if all_doc_ids is None:
            raise ValueError("Para NOT, debe proveer all_doc_ids.")
        result = set(all_doc_ids) - postings_lists[0]
    else:
        raise ValueError("Operación no soportada. Use 'AND', 'OR', o 'NOT'.")

    return pd.DataFrame({'ID': sorted(result)})

In [13]:
# Ejemplo de uso para AND, NOT:
all_doc_ids = df_documents['ID'].tolist()

print("Consulta AND:", boolean_query(inverted_index, 'AND', ['create search'], all_doc_ids=all_doc_ids))
print(100*"*")
print("Consulta NOT:", boolean_query(inverted_index, 'NOT', ['love'], all_doc_ids=all_doc_ids))

Consulta AND:      ID
0  d065
1  d158
2  d177
3  d216
4  d227
5  d280
6  d314
****************************************************************************************************
Consulta NOT:        ID
0    d001
1    d002
2    d003
3    d005
4    d006
..    ...
292  d327
293  d328
294  d329
295  d330
296  d331

[297 rows x 1 columns]


**ENUNCIADO: [5p] Para cada una de las 35 consultas en el conjunto de datos, recupere los documentos utilizando consultas binarias AND (i.e. termino_1 AND termino_2 AND termino_3…). Escriba un archivo (BSII-ANDqueries_results) con los resultados siguiendo el mismo formato que "relevance-judgments":**

#### 1. Extraer las queries usando la función ingest_naf_documents

In [14]:
def ingest_queries(directory):
    df_queries = ingest_naf_documents(directory)
    return df_queries[['ID', 'raw_text']]

df_queries = ingest_queries(QUERIES_DIRECTORY_PATH)

# Ordernar por ID
df_queries = df_queries.sort_values('ID')
# Ver las primeras queries
df_queries.head()

,ID,raw_text
23,q01,Fabrication of music instruments
29,q02,famous German poetry
30,q03,Romanticism
13,q04,University of Edinburgh research
7,q06,bridge construction


#### 2. Recuperar documentos para cada query usando boolean_query (AND)

In [15]:
all_doc_ids = df_documents['ID'].tolist()
results = []

for _, row in df_queries.iterrows():
    query_id = row['ID']
    query_text = row['raw_text']
    # Usar boolean_query con preprocesamiento
    docs_df = boolean_query(inverted_index, 'AND', [query_text], all_doc_ids=all_doc_ids)
    doc_list = ','.join(docs_df['ID'].tolist())
    results.append(f"{query_id}\t{doc_list}")

#### 3. Escribir el archivo de resultados

In [16]:
with open("BSII-ANDqueries_results.tsv", "w") as f:
    for line in results:
        f.write(line + "\n")